In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import signal
import glob

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from cartopy.feature import ShapelyFeature
import numpy as np
import xarray as xr
import cartopy as cart
import cartopy.io.shapereader as shpreader
from mpl_toolkits.basemap import Basemap
from xclim import ensembles
import rioxarray
import pathlib
import pymannkendall as mk


import geopandas
from shapely.geometry import mapping

In [ ]:
ST=geopandas.read_file('/mnt/CEPH_PROJECTS/FACT_CLIMAX/OBS_DATA/shape_region/TrentinoAltoAdige.shp')

ST=ST[ST.ID == 47046.0]

ST=ST.to_crs("EPSG:3035")

#ST.crs

In [ ]:
from glob import glob
import xarray as xr

def read_netcdfs(files,dim):
    paths = sorted(glob(files))
    datasets = [xr.open_dataset(p) for p in paths]
    combined = xr.concat(datasets, dim = 'realization')
    return combined

#rioxarray.open_rasterio,masked=True

var = 'spi12'
Var = ['spi12']
rcp = ['rcp85']



for i in Var:
    for j in rcp:
        ind = read_netcdfs('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/INDICES/{}/{}/{}_*.nc'.format(i,j,i),'realization')
    
index = ind.median(dim='realization')

In [ ]:
#ind =ind.mean(dim='bnds')   # remove bnds- only for some variables

In [ ]:
ind = ind.rio.write_crs({'init': 'EPSG:3035'})
#ind.rio.crs

In [ ]:
clipped_ind = ind.rio.clip(ST.geometry.apply(mapping), ST.crs, drop=True,invert=False)
#all_touched=True

In [ ]:
# digital elevation model: Copernicus EU-Dem v1.1
dem_t=xr.open_dataset('/mnt/CEPH_PROJECTS/FACT_CLIMAX/DEM/eu_dem_v11_stt.nc')

time=pd.date_range('1970-01-01','2100-12-01',freq='M')

time=xr.DataArray(time,[('time',time)])

dem_t=dem_t.Band1.to_dataset().rename({'Band1': 'elevation'})

time_n, dem_t = xr.broadcast(time,dem_t)
del time_n

In [ ]:
dem_t = dem_t.rio.write_crs({'init': 'EPSG:3035'})

In [ ]:
clipped_dem = dem_t.rio.clip(ST.geometry.apply(mapping), ST.crs, drop=True,invert=False)

In [ ]:
clipped_ind=clipped_ind.sel(time=slice('1970-02-01','2100-12-01'))

In [ ]:
dem_t['time']=clipped_ind.time

In [ ]:
#clipped_ind = clipped_ind.isel(time=slice(1,133))  # only for some variables

In [ ]:
#Create mask from DEM

D= []
for k, _ in enumerate(clipped_ind.realization):
    #print(i)
    dem_mask=np.ma.masked_where(clipped_dem['elevation']<1500,clipped_dem['elevation'])
    D.append(clipped_ind.isel(realization=k).transpose('time','x','y').where(dem_mask.mask))
    
masked_ind=xr.concat(D,dim='realization')

In [ ]:
VALID_PERIOD = slice('1981-12-31', '2010-12-31')
F1_PERIOD = slice('2041-12-31', '2070-12-31')
F2_PERIOD = slice('2071-12-31', '2100-12-31')

H = clipped_ind.sel(time=VALID_PERIOD).median(dim=('realization')).mean('time')
F1 = clipped_ind.sel(time=F1_PERIOD).median(dim=('realization')).mean('time')
F2 = clipped_ind.sel(time=F2_PERIOD).median(dim=('realization')).mean('time')

#for masked values
H_m = masked_ind.sel(time=VALID_PERIOD).median(dim=('realization')).mean('time')
F1_m = masked_ind.sel(time=F1_PERIOD).median(dim=('realization')).mean('time')
F2_m = masked_ind.sel(time=F2_PERIOD).median(dim=('realization')).mean('time')


d1=(F1-H)
d2=(F2-H)

d1_m=(F1_m-H_m)
d2_m=(F2_m-H_m)

In [ ]:
print(d2.max(),d2.min())

In [ ]:
NAMES = {'mat': 'Mean annual temperature', 'frd': 'Frost days', 'trn': 'Tropical nights', 'spr':'Summer total precipitation', 'wpr':'Winter total precipitation', '5dmax':'Maximum 5-day precipitation',
        'hpr95wet':'Heavy precipitation (95-th P / wet days)', 'hpf95wet':'Heavy precipitation frequency (95-th P / wet days)', 'spi12':'SPI-12', 'spei12':'SPEI-12'} 

UNITS = {'mat': 'degree Celsius', 'frd': 'Number of days', 'trn': 'Number of days', 'spr':'mm', 'wpr':'mm', '5dmax':'mm',
        'hpr95wet':'mm', 'hpf95wet':'Number of days', 'spi12':'Index', 'spei12':'Index'} 


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,3), sharex=True, sharey=True)

vmin=-2
vmax=2
x = np.linspace(vmin, vmax,11)

variable=d1[var]
variable2=d2[var]

image=ax[0].contourf(variable.values,10,cmap='Blues')
image2=ax[1].contourf(variable2.values,10,cmap='Blues')

cbar_ax_predictand = fig.add_axes([ax[0].get_position().x0+0.173, ax[0].get_position().y0 - 0.2,
                                   ax[-1].get_position().x0 - ax[0].get_position().x0,
                                   0.05])

cbar=fig.colorbar(image,cax=cbar_ax_predictand,orientation='horizontal',extend='both',shrink=0.60,format='%.1f')
#cbar.set_label(UNITS[var])
plt.title(NAMES[var])  
ax[0].set_title('2041-2070')
ax[1].set_title('2071-2100')

for axes in ax.flat:
    axes.set(xlabel='lon', ylabel='lat')
    
for axes in ax.flat:
    axes.label_outer()
        
fig.tight_layout()


fig.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/f1_f2_maps_{}_{}.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')

In [ ]:
#Ensamble statistics:
index_stats = ensembles.ensemble_mean_std_max_min(clipped_ind.mean(dim=['y','x']))
index_perc = ensembles.ensemble_percentiles(clipped_ind.mean(dim=['y','x']), values=[25, 50, 75], split=False)

#for masked areas:
index_stats_m = ensembles.ensemble_mean_std_max_min(masked_ind.mean(dim=['x','y']))
index_perc_m = ensembles.ensemble_percentiles(masked_ind.mean(dim=['x','y']), values=[25, 50, 75], split=False)


In [ ]:
##Trend calculation for clipped area
x=np.arange(0,len(clipped_ind.time.dt.year.values))
z = np.polyfit(x, clipped_ind[var].mean(['x','y']).median(['realization']).fillna(0.0), 1)
p = np.poly1d(z)

#Plotting
plt.plot(clipped_ind.time.dt.year.values,clipped_ind[var].mean(['x','y']).median(['realization']))
plt.plot(clipped_ind.time.dt.year.values,p(x),"r--")
plt.ylabel(NAMES[var])
plt.xlabel('Year')

In [ ]:
##Trend calculation for clipped and masked area
xi=np.arange(0,len(masked_ind.time.dt.year.values))
zi = np.polyfit(xi, masked_ind[var].mean(['x','y']).median(['realization']).fillna(0.0), 1)
pi = np.poly1d(zi)

#Plotting
plt.plot(masked_ind.time.dt.year.values,masked_ind[var].mean(['x','y']).median(['realization']))
plt.plot(masked_ind.time.dt.year.values,pi(xi),"r--")
plt.ylabel(var)
plt.xlabel('Year')

In [ ]:
##Plotting

#.isel(time=slice(1,131))  - only for 5dmax

stat = index_stats_m
perc = index_perc_m
data = masked_ind
fig, ax = plt.subplots(figsize=(10,7))
ax.fill_between(stat.time.dt.year.values, stat['spi12_min'].squeeze(), stat['spi12_max'].squeeze(), alpha=0.3, label='Min-Max')
ax.fill_between(stat.time.dt.year.values, perc[var].squeeze().sel(percentiles=25), perc[var].squeeze().sel(percentiles=75), alpha=0.5, label='Perc. 25-75')
ax._get_lines.get_next_color()  # Hack to get different line
ax._get_lines.get_next_color()

ax.plot(data.time.dt.year.values,data[var].mean(['x','y']).median(['realization']), linewidth=1.2, label='Ensemble median')
ax.plot(data.time.dt.year.values,pi(xi),"b--",linewidth=1,label='Trend')
ax.legend(loc='best')
plt.title(NAMES[var])
plt.ylabel(NAMES[var])
plt.xlabel('Year')

#plt.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/stat_{}_{}.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 6))

clrs = ['blue' if (x >=0) else 'red' for x in masked_ind[var].mean(['x','y']).median(['realization']).fillna(0.0).values]

axes.bar(masked_ind.time.dt.year.values,masked_ind[var].mean(['x','y']).median(['realization']).fillna(0.0).values, color=clrs, width=0.31)
axes.axhline(y = -1, color = 'grey', linestyle = 'dashed',linewidth=1,alpha=0.3)
axes.axhline(y = 1, color = 'grey', linestyle = 'dashed',linewidth=1,alpha=0.3)
axes.axhline(y = -1.5, color = 'grey', linestyle = 'dashed',linewidth=1,alpha=0.9)
axes.axhline(y = 1.5, color = 'grey', linestyle = 'dashed',linewidth=1,alpha=0.9)
axes.set_ylim(-3,3)
axes.set_ylabel(NAMES[var])
axes.set_xlabel('Year')
axes.set_title(NAMES[var])


fig.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/{}_{}_m.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')

In [ ]:
df1 = masked_ind[var].mean(['x','y']).median(['realization']).fillna(0.0)[(np.where((masked_ind[var].mean(['x','y']).median(['realization']).fillna(0.0).values<=-1) ))]    # moderate event
df1 = df1.groupby(df1.time.dt.year).count()

In [ ]:
df1.drop_vars('lambert_azimuthal_equal_area').to_dataframe().to_csv('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/n_dr_events_{}_{}_m.csv'.format(NAMES[var],rcp))

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
axes.bar(df1.year,df1, width=0.7)
axes.set_ylabel('Number of dry events')
axes.set_title('Number of dry events')
axes.legend(loc='best')

axes.set_ylim(0,3)


fig.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/N_dr_events_{}_{}_v1_m.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
axes.plot(df1.year,df1,linestyle="",marker='o',c='purple',label='moderate')

axes.set_ylabel('Number of dry events')
axes.set_title('Number of dry events')
axes.legend(loc='best')

#fig.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/N_dr_events_{}_{}_v2.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')

In [ ]:
vari=clipped_ind[var].mean(['x','y']).median(['realization']).fillna(0.0).load()
dry_days=vari[np.where((vari.values<=-0.5))]

period=[]
values=[]

f=range(0,len(vari.time)-3)
i=0

while i<=len(f):    
    A=vari.isel(time=i).values
    B=vari.isel(time=i+1).values
    C=vari.isel(time=i+2).values
    
    if A <=-1 and B in dry_days.values and C in dry_days.values:
        while vari.isel(time=i).values in dry_days.values:
            period.append(vari.isel(time=i).time.values)

            values.append(vari.isel(time=i).values)
            i+=1
    i+=1

In [ ]:
time=pd.DataFrame(period,columns=['Time'])
values=pd.DataFrame(values,columns=['Values'])

time['id'] = range(0, len(time))
values['id'] = range(0, len(time))
dr=pd.merge(time,values,on='id')
dr=dr.set_index('Time')
time_all=pd.DataFrame(vari.time.values,columns=['Time'])

time_all['id'] = range(0, len(time_all))
time_all=time_all.set_index('Time')


dr_ts=pd.merge(dr, time_all, left_index=True, right_index=True, how="outer").reset_index('Time')
dr_ts=dr_ts[['Time','Values']]
dr_ts = dr_ts.replace(np.nan,  999.)  #"%.1f" %


#Calculating magnitude(sum) of a single drought event

ar=dr_ts.Values.array
ar_r=range(0,len(ar))
dr_sum=[]
i=0
while i<=len(ar_r)-3:
    su=0
    if ar[i] <=0:
        while ar[i] <=0:
            su += ar[i]
            i+=1
    i+=1
    dr_sum.append(su)
dr_magn=np.asarray(list(dict.fromkeys(dr_sum))[1:])


#Looking for a starting date of a single drought event

k=2
dates_strt=[]
while k <= len(ar_r)-2:
    if ar[k]<=0 and ar[k-1]>=0:
        while ar[k]<=0 and ar[k-1]>=0:
            dates_strt.append(dr_ts.Time.array[k])
            k+=1
    k+=1
time_strt=pd.DataFrame(dates_strt,columns=['Time_strt'])
Magnitude=pd.DataFrame(dr_magn,columns=['Magnitude'])


#Looking for an ending date of a single drought event

z=0
dates_nd=[]
while z <= len(ar_r)-2:    
    if ar[z]<=0 and ar[z+1]>=0:
        while ar[z]<=0 and ar[z+1]>=0:
            dates_nd.append(dr_ts.Time.array[z])
            z+=1
    z+=1
time_nd=pd.DataFrame(dates_nd,columns=['Time_nd'])


#Calculate severity of a drought

from operator import attrgetter

#drought duration
duration=(time_nd['Time_nd'].apply(lambda x: x+np.timedelta64(1, 'M')).apply(lambda x: x.to_period('M'))
      - time_strt['Time_strt'].apply(lambda x: x.to_period('M'))).apply(attrgetter('n')) 

dr_sev=np.asarray(dr_magn) / duration
Severity=pd.DataFrame(dr_sev.values,columns=['Severity'])
Duration=pd.DataFrame(duration.values,columns=['Duration'])

In [ ]:
data_frames=[time_strt,time_nd,Duration,Severity,Magnitude]
droughts_df=pd.concat(data_frames,axis=1).rename(columns={"Time_strt": "Start date", "Time_nd": "End date"}).round({'Severity': 1, 'Magnitude': 1})
droughts_df

In [ ]:
droughts_df.to_csv('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/droughts_{}_{}_m.csv'.format(NAMES[var],rcp))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 4))
axes = axes.flatten()

t_s=np.arange(0,len(droughts_df['Start date']))
z_s=np.polyfit(t_s, droughts_df['Severity'].values, 1)
p_s=np.poly1d(z_s)

axes[0].plot(droughts_df['Start date'],droughts_df['Severity'],c='black')
##axes[0].plot(droughts_df['Time_strt'].values,p_s(t_s),"r")
axes[0].set_title('Severity of drought events')
trend, h, p, z, Tau, s, var_s, slope, intercept= mk.original_test(droughts_df['Severity'].values)
axes[0].text(x=30000 , y=-1.7, s='p-value = {:.2f}, trend - {}'.format(p.item(),trend), fontsize=12, ha='right')
axes[0].set_ylabel(NAMES[var])
axes[0].set_xlabel('Year')

t_m=np.arange(0,len(droughts_df['Start date']))
z_m=np.polyfit(t_m, droughts_df['Magnitude'].values, 1)
p_m=np.poly1d(z_m)
#res = stats.theilslopes(country.groupby(country.time.dt.month)[j].mean(['x','y']).values, p_s(x_s), 0.95)

axes[1].plot(droughts_df['Start date'],droughts_df['Magnitude'],c='black')
##axes[1].plot(droughts_df['Time_strt'].values,p_m(t_m),"r")
axes[1].set_title('Magnitude of drought events')
trend, h, p, z, Tau, s, var_s, slope, intercept= mk.original_test(droughts_df['Magnitude'].values)
axes[1].text(x=30000 , y=-120, s='p-value = {:.2f}, trend - {}'.format(p.item(), trend), fontsize=12, ha='right')
axes[1].set_ylabel(NAMES[var])
axes[1].set_xlabel('Year')

fig.savefig('/mnt/CEPH_PROJECTS/FACT_CLIMAX/CORDEX-Adjust/plots/sev_magn_{}_{}_m.png'.format(NAMES[var],rcp), dpi=300, bbox_inches='tight')